# Week 6: Data Retrieval and Processing

1. Retrieve your Milestone 1 output to start the retrieval process.
2. Get the total number of stored records:


In [7]:
from web3 import Web3

# Connect to local Ethereum node (update the provider if needed)
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:7545'))

# Replace with your contract's ABI and address
contract_abi = [
	{
		"inputs": [],
		"name": "getTotalRecords",
		"outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
		"stateMutability": "view",
		"type": "function"
	}
]
contract_address = w3.to_checksum_address('0x069af8b9480dcb0001eb618a72d1ff57c3e350de')  # <-- insert your contract address here

contract = w3.eth.contract(address=contract_address, abi=contract_abi)

total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 195


3. Fetch all stored IoT data and structure it in a DataFrame:

In [8]:
# Only add if not already in contract_abi
if not any(f.get("name") == "getRecord" for f in contract_abi):
    contract_abi.append(get_record_abi)
    contract = w3.eth.contract(address=contract_address, abi=contract_abi)

# Retrieve all IoT records
data = []
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "timestamp": record[0],
        "device_id": record[1],
        "data_type": record[2],
        "data_value": record[3]
    })

# Convert to a DataFrame
df = pd.DataFrame(data)

# Convert timestamp to readable format
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

# Display first few records
print(df.head())

            timestamp device_id    data_type data_value
0 2025-06-19 05:45:08    DVC776  temperature       1.97
1 2025-06-19 05:45:09    DVC684  temperature      14.29
2 2025-06-19 05:45:10    DVC379  temperature      15.78
3 2025-06-19 05:45:11    DVC605  temperature       9.33
4 2025-06-19 05:45:12    DVC621  temperature      16.98


4. After the data is structured and cleaned, it is preprocessed for further analysis. Convert numerical values where applicable:
 - Some IoT sensor readings may contain units or text (e.g., "22.5°C", "50% humidity", "15.3 kWh"). You need to extract the numerical values to make the data usable.
 ##### Identify missing values.
 * If missing values are minor, replace them with 0.
 * If missing values are significant, use the mean or median of the column.


In [9]:
import numpy as np


# Extract numeric values from 'data_value' where applicable
df["numeric_value"] = df["data_value"].str.extract(r'(\d+\.?\d*)').astype(float)


# Handle missing values (if any)
df.fillna(0, inplace=True)



# Display cleaned data
print(df.head())


            timestamp device_id    data_type data_value  numeric_value
0 2025-06-19 05:45:08    DVC776  temperature       1.97           1.97
1 2025-06-19 05:45:09    DVC684  temperature      14.29          14.29
2 2025-06-19 05:45:10    DVC379  temperature      15.78          15.78
3 2025-06-19 05:45:11    DVC605  temperature       9.33           9.33
4 2025-06-19 05:45:12    DVC621  temperature      16.98          16.98


6. Save the DataFrame as a CSV file:

In [10]:
# Save cleaned IoT data to a CSV file
df.to_csv("cleaned_iot_data.csv", index=False)


print("✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv")

✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv
